# Задача

In [2]:
import numpy as np
import pandas as pd
import sklearn.linear_model as lm
import statsmodels.api as sm
import patsy as pt
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image
from math import sqrt
import statistics
from scipy import stats

In [3]:
data = pd.read_csv('zadacha.csv', ';')
df = data.drop('Family', axis='columns')
df.head()

,S,Y,W
0,3.0,40,60
1,6.0,55,36
2,5.0,45,36
3,3.5,30,15
4,1.5,30,90


## А.

In [21]:
X = df.drop(df.columns[0], axis='columns')
y = df['S']
reg = lm.LinearRegression()
reg.fit(X, y)
#X = np.hstack((np.ones((np.shape(X)[0], 1)), X))

coeffs = list()
coeffs.append(reg.intercept_)
coeffs.extend(reg.coef_)

print('Полученные коэффициенты модели:\n')
print('intercept : ', coeffs[0])
for item in zip(df.columns[1:], coeffs[1:]):
    print(item[0], ' : ', item[1])

Полученные коэффициенты модели:

intercept :  0.2787386557937661
Y  :  0.12287912666052875
W  :  -0.02940725152352143


## Б.

In [22]:
def predict(Y, W):
    return coeffs[0] + Y * coeffs[1] + W * coeffs[2]

In [23]:
round(predict(40, 25), 1)

4.5

## В.

Согласно построенному уравнению регрессии, изменение накоплений будут линейно зависеть от изменения дохода при прочих равных.
Коэффициентом пропорциональности между ними как раз является соответствующий коэффициент регрессии.

In [24]:
print('Возрастут на %s тыс. руб.' % round(coeffs[1] * 10, 1))

Возрастут на 1.2 тыс. руб.


## Г.

Аналогично предыдущему пункту, изменение по двум переменным также пропорционально каждой с соответствующими коэффициентами регрессии.

In [25]:
print('Возрастут на %s тыс. руб.' % round(coeffs[1] * 5 + coeffs[2] * 15, 1))

Возрастут на 0.2 тыс. руб.


## Д.

In [34]:
Xx = np.hstack((np.ones((np.shape(X)[0], 1)), X))
y_pred = Xx.dot(coeffs)
#print(y_pred)
#print(reg.predict(X))
S2n = sum((a - b)**2 for a, b in zip(y, y_pred))
print('Сумма квадратов остатков:\t%s\n' % S2n)

S2 = S2n/(y.shape[0]-2)
print('Выборочная дисперсия:\t%s\n' % S2)

[3.42946863 5.97842957 4.7496383  3.52400368 1.31845982]
[3.42946863 5.97842957 4.7496383  3.52400368 1.31845982]
Сумма квадратов остатков:	0.28112258319084565

Выборочная дисперсия:	0.09370752773028189



# ДЗ 2.

In [38]:
import statsmodels.api as sm

model = sm.OLS(y,Xx)
results = model.fit()

coeffs = results.params
print(coeffs)

cov_matrix = results.normalized_cov_params
cov_matrix

const    0.278739
x1       0.122879
x2      -0.029407
dtype: float64


,const,x1,x2
const,5.691635,-0.107392,-0.025231
x1,-0.107392,0.002399,0.000241
x2,-0.025231,0.000241,0.000329


### В каждом пункте нужно проверить ограниечение 3.40 из учебника.

In [48]:
def mat(H, x, beta): 
    A = H.T@np.linalg.inv((H@np.linalg.inv((x.T@x))@H.T))@H
    return A

e_ = (y - reg.predict(X)).T@(y - reg.predict(X))

H23 = np.array([[1, 0], [0, 1]])
H2 = np.array([[ 1, 0]])
H3 = np.array([[ 0, 1]])
H12 = np.array([[1, 0, 0], [0, 1, 0]])

In [49]:
np.array(X)

array([[40, 60],
       [55, 36],
       [45, 36],
       [30, 15],
       [30, 90]])

## А.

In [79]:
alpha = 0.05

reg.fit(X, y)
A = mat(H23, X, reg.coef_)
c = (y - reg.predict(X)).T@(y - reg.predict(X))*2/3

print('\n1. Доверительное множество для b2 и b3:')
print('c = \n', round(c, 4))
print('A = \n', A)


1. Доверительное множество для b2 и b3:
c = 
 0.1874
A = 
 [[ 8450.  9150.]
 [ 9150. 14517.]]


$(\hat{\beta} - \beta)'A (\hat{\beta} - \beta) \in [0; 19.16\cdot c],$

$19.16$ = квантиль уровня $0.05$ у $F_{2,3}$.

In [91]:
#se = (cov_matrix.iloc[1,1])**0.5
#bound = se * stats.t.ppf(1-alpha/2., df.shape[0]-3)

reg.fit(X, y)
A = mat(H2, X, reg.coef_)
c = (y - reg.predict(X)).T@(y - reg.predict(X))*1/3
t_up = reg.coef_[1] + sqrt(216)*c/A[0,0]
t_lo = reg.coef_[1] - sqrt(216)*c/A[0,0]

#print('2. Доверительный интервал для b2:\t', round(coeffs[1] - bound, 3), '\t', round(coeffs[1] + bound, 3))
print('\n2. Доверительный интервал для b2:\t', round(t_lo, 4), '\t', round(t_up, 4))

#se = (cov_matrix.iloc[2,2])**0.5
#bound = se * stats.t.ppf(1-alpha/2., df.shape[0]-3)

reg.fit(X, y)
A = mat(H3, X, reg.coef_)
c = (y - reg.predict(X)).T@(y - reg.predict(X))*1/3
t_up = reg.coef_[1] + sqrt(216)*c/A[1,1]
t_lo = reg.coef_[1] - sqrt(216)*c/A[1,1]

#print('3. Доверительный интервал для b3:\t', round(coeffs[2] - bound, 3), '\t', round(coeffs[2] + bound, 3))
print('\n3. Доверительный интервал для b3:\t', round(t_lo, 4), '\t', round(t_up, 4))

reg.fit(Xx, y)
A = mat(H12, Xx, reg.coef_)
c = (y - reg.predict(Xx)).T@(y - reg.predict(Xx))*2/3

print('\n4. Доверительное множество для b1 и b2:')
print('c = \n', round(c, 4))
print('A = \n', A)


2. Доверительный интервал для b2:	 -0.0299 	 -0.0289

3. Доверительный интервал для b3:	 -0.0297 	 -0.0291

4. Доверительное множество для b1 и b2:
c = 
 0.1874
A = 
 [[1.13081215e+00 5.06199628e+01 0.00000000e+00]
 [5.06199628e+01 2.68279603e+03 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]]


$(\hat{\beta} - \beta)'A (\hat{\beta} - \beta) \in [0; 19.16\cdot c],$

$19.16$ = квантиль уровня $0.05$ у $F_{2,3}$.

## Б.

Согласно полученным доверительным интервалам, на уровне значимости 0.05:
1. Отвергается гипотеза о том, что $\beta_2 = 0, \beta_3 = 0$ 
2. Нельзя отвергнуть гипотезу о том, что $\beta_3 = 0$,
3. Отвергается гипотеза о том, что $\beta_2 = 0$,
4. Отвергается гипотеза о том, что $\beta_2 = 1$,
5. Отвергается гипотеза о том, что $\beta_2 = 1.57$,
6. 

In [87]:
H = np.array([[ 1, -5]])
reg.fit(X, y)
A = mat(H, X, reg.coef_)
c = (y - reg.predict(X)).T@(y - reg.predict(X))*1/3
print('c = \n', round(c, 4))
print('A = \n', A)

c = 
 0.0937
A = 
 [[ 122.75512423 -613.77562116]
 [-613.77562116 3068.87810582]]


Даже при $\beta_2, \beta_3$, минимизирующих статистику, $(\hat{\beta} - \beta)'A (\hat{\beta} - \beta)$ всё равно достигает экстремальных значений:

In [88]:
x = (-reg.coef_[0]*(A[0,0]+A[1,0]) - reg.coef_[1]*(A[0,1]+A[1,1]))/((A[0,0]+A[1,0]) -5*(A[0,1]+A[1,1]))
coef = reg.coef_ - [ x, -5*x]
print(coef.T@A@coef, 19.16*c)

35.7729617928818 1.795436231312197


Поэтому отвергается гипотеза о том, что $\beta_2 = -5\beta_3$

## В.

In [95]:
Y = 30
W = 52.5
print('Семья Икс:\t', round(predict(Y, W), 1))

Y = 30
W = 15
print('Семья 4:\t', round(predict(Y, W), 1))

Y = 30
W = 90
print('Семья 5:\t', round(predict(Y, W), 1))

data.iloc[3:]

Семья Икс:	 2.4
Семья 4:	 3.5
Семья 5:	 1.3


,Family,S,Y,W
3,4,3.5,30,15
4,5,1.5,30,90


Семья Икс может рассматриваться как средняя междй семьями 4 и 5 в том плане, что при равных доходах ее имущество равно среднему арифметическому имуществ семей 4 и 5.
Рассчитанные моделью накопления семьи Икс (2.4) не равны среднему арифметическому фактических накоплений семей 4 и 5. Дело в том, что этого можно ожидать от линейной модели в отношении таких же рассчитанных для семей 4 и 5 величин. Если подсчитать накопления семей 4 и 5 с помощью модели (3.5, 1.3), они будут отличаться от фактических (3.5, 1.5), но уже будут отвечать этому свойству – рассчитанные накопления семьи Икс будут равны среднему арифметическому рассчитанных накоплений семей 4 и 5.

In [94]:
reg.fit(X, y)
t_up = reg.intercept_ + reg.coef_[0]*30 + reg.coef_[1]*52.5+1.95
t_lo = reg.intercept_ + reg.coef_[0]*30 + reg.coef_[1]*52.5-1.95
print('\nДоверительное множество для накоплений этой семьи:', round(t_lo, 4), round(t_up, 4))


Доверительное множество для накоплений этой семьи: 0.4712 4.3712
